In [40]:
import sys

sys.path.append("../src/stong_core_cython")
sys.path.append("../src/stong_core_cython/md")

In [41]:
import numpy as np
import pandas as pd
import hartree_fock
import basis_data
import importlib
import matrix
import two_electron
from scipy.optimize import minimize

importlib.reload(hartree_fock)
importlib.reload(basis_data)
importlib.reload(matrix)
importlib.reload(two_electron)

from atom import *
from hartree_fock import *
from basis_data import *
from matrix import *
from two_electron import *

In [42]:
df = pd.read_csv("../examples/make_base/sto_3g.csv")

In [43]:
zetas = [1, 1, 1]
basis_data = {"H": create_basis_data(df, zetas, 3, 1)}
molecule = Molecule(
    [Atom("H", np.array([0.0, 0.0, 0.0])), Atom("H", np.array([0.0, 0.0, 0.74]))]
)
basis_functions = create_basis_function(molecule, basis_data)

In [44]:
len(basis_functions)

18

In [45]:
basis_functions[0].lmn

<MemoryView of 'ndarray' at 0x7cf965e94ee0>

In [49]:
get_Tmat(basis_functions)

TypeError: a bytes-like object is required, not 'tuple'

In [51]:
T(basis_functions[0], basis_functions[0])

TypeError: a bytes-like object is required, not 'tuple'

In [50]:
get_Smat(basis_functions)

array([[ 1.        ,  0.86692628,  0.        ,  0.        ,  0.        ,
         0.63288479,  0.        ,  0.        ,  0.        ,  0.91730486,
         0.83790223,  0.        ,  0.        , -0.33946468,  0.63044581,
         0.        ,  0.        , -0.26107862],
       [ 0.86692628,  1.        ,  0.        ,  0.        ,  0.        ,
         0.91235994,  0.        ,  0.        ,  0.        ,  0.83790223,
         0.97152938,  0.        ,  0.        , -0.20914309,  0.89637127,
         0.        ,  0.        , -0.22729131],
       [ 0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.91374855,  0.        ,  0.        ,  0.        ,
         0.        ,  0.94758098,  0.        ,  0.        ,  0.        ,
         0.88387617,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  0.        ,  0.91374855,  0.        ,  0.        ,
         0.        ,  0.        ,  0.94758098,  0.   

In [46]:
print(
    ERI(basis_functions[0], basis_functions[17], basis_functions[0], basis_functions[1])
)
print(
    ERI(basis_functions[17], basis_functions[0], basis_functions[0], basis_functions[1])
)
print(
    ERI(basis_functions[0], basis_functions[1], basis_functions[0], basis_functions[17])
)

-0.1164940321387546
-0.11649403213875459
-0.1164940321387546


In [47]:
a = basis_functions[0]
b = basis_functions[17]
c = basis_functions[0]
d = basis_functions[1]
i, j, k, l = 0, 2, 0, 0
electron_repulsion(
    a.exps[i],
    a.lmn,
    a.origin,
    b.exps[j],
    b.lmn,
    b.origin,
    c.exps[k],
    c.lmn,
    c.origin,
    d.exps[l],
    d.lmn,
    d.origin,
)

-66.71035054567223

In [48]:
i, j, k, l = 2, 0, 0, 0
electron_repulsion(
    a.exps[i],
    a.lmn,
    a.origin,
    b.exps[j],
    b.lmn,
    b.origin,
    c.exps[k],
    c.lmn,
    c.origin,
    d.exps[l],
    d.lmn,
    d.origin,
)

-36.584622160798745